In [3]:
import json
import random

# Load the playlist data from a JSON file
with open('Vetting_Playlist.json', 'r') as f:
    playlist_data = json.load(f)

# Check if the 'songs' key exists in the loaded JSON data
if 'songs' in playlist_data:
    # Extract the current songs from the playlist
    current_songs = playlist_data['songs']

    # Define a function to recommend three songs based on the current playlist
    def recommend_songs(current_songs, playlist_data):
        recommended_songs = []
        # Here, you would implement your recommendation logic
        # For simplicity, let's randomly select three songs from the playlist
        recommended_songs = random.sample(playlist_data['songs'], min(3, len(playlist_data['songs'])))
        return recommended_songs

    # Get the recommended songs
    recommended_songs = recommend_songs(current_songs, playlist_data)

    # Print the recommended songs
    print("Recommended Songs:")
    for song in recommended_songs:
        print(song['title'], '-', song['artist'])
else:
    print("The 'songs' key does not exist in the playlist data.")




The 'songs' key does not exist in the playlist data.
